In [1703]:
import numpy as np
import pandas as pd
# import sklearn.datasets as data

In [1704]:
df = pd.read_csv("GroupExam2024_data_and_documentation/elektronisk-rapportering-ers-2018-fangstmelding-dca-simple.csv", sep=';', decimal=',')
df.head()
print(df.shape)


(305434, 45)


Dropped Meldingstidspunkt, Starttidspunkt, Art - FDIR and strings from dataset due to how the data scaling generaly dislikes strings!
After dropping stings the scores went from -11% to 20%! (Both test and training) 

Maybe use one-hot encoding to include it anyway?

In [1705]:
#Sile ut informasjon:
df = df[(df['Art - FDIR (kode)'] == 1022) | (df['Art - FDIR (kode)'] == 1027)] # Fiske som fanger hyse eller torsk. "751 - lodde, 1022 Torsk"
area = [7, 8, 28, 30, 40]
df = df[(df['Redskap FDIR (kode)'] == 51) & (df['Hovedområde start (kode)'].isin(area))] # Fiske med bunntrål - 51, snurrevad - 11
df['Bruttotonnasje Kombinert'] = df['Bruttotonnasje 1969'].fillna(df['Bruttotonnasje annen']) # Slår sammen kolonnene.
# Dropper unødvendig informasjon. Tabellen inneholder masse duplikater. som egen kolonne for navn og for kode
# Forskjellige klassifiseringsmåter. Holder på FDIR sine.

# Normal pre-processing som vi gjorde på alt annet
df.drop(columns = ['Meldingsdato', 
                   'Meldingsklokkeslett', 
                   'Startdato',  
                   'Hovedområde start', 
                   'Lokasjon start (kode)', 
                   'Stoppdato', 
                   'Stoppklokkeslett', 
                   'Fangstår', 
                   'Hovedområde stopp (kode)', 
                   'Hovedområde stopp', 
                   'Lokasjon stopp (kode)', 
                   'Redskap FAO (kode)', 
                   'Redskap FAO', 
                   'Redskap FDIR', 
                   'Hovedart FAO (kode)', 
                   'Hovedart FAO', 
                   'Art FAO (kode)',
                   'Art FAO',
                   'Art - gruppe', 
                   'Lengdegruppe', 
                   'Bredde', 
                   'Fartøylengde',
                   'Bruttotonnasje 1969',
                   'Bruttotonnasje annen',

                    #kolonner fjernet for MLP regressor
                   'Meldingstidspunkt',
                   'Starttidspunkt',
                   'Startklokkeslett',
                   'Stopptidspunkt',
                   'Art - FDIR',
                   'Melding ID',
                   'Havdybde start',# Filter and remove incorrect values
                   'Havdybde stopp',# Filter and remove incorrect values
                   'Redskap FDIR (kode)', #Fjernet grunnet gjentagene data som ikke gav noe verdi
                    'Hovedart - FDIR (kode)', #Fjernet grunnet gjentagene data som ikke gav noe verdi
                    'Art - FDIR (kode)', #Fjernet grunnet gjentagene data som ikke gav noe verdi
                    'Art - gruppe (kode)', #Fjernet grunnet gjentagene data som ikke gav noe verdi
    ], inplace=True)

# Gruppere bruttotonnasje:
bins = [x for x in range(0, 4800, 100)] # Definerer gruppene
labels = range(1, len(bins))  # nummererer gruppene.
# Categorize 'Bruttotonnasje Kombinert' into the defined bins
df['Bruttotonnasje Gruppert'] = pd.cut(df['Bruttotonnasje Kombinert'], bins=bins, labels=labels)
# Teller grupperingene
df['Bruttotonnasje Gruppert'].value_counts()

Bruttotonnasje Gruppert
6     992
3     950
2     618
21    465
4     433
35    312
14    305
32    261
5     257
25    233
1     210
12    191
16    186
15    181
40    164
38     78
26     50
7      49
23     45
19     44
45     17
17      8
42      5
44      0
46      0
41      0
33      0
37      0
36      0
43      0
34      0
39      0
24      0
31      0
30      0
29      0
28      0
27      0
22      0
20      0
18      0
13      0
11      0
10      0
9       0
8       0
47      0
Name: count, dtype: int64

In [1706]:
df.head()

,Startposisjon bredde,Startposisjon lengde,Hovedområde start (kode),Varighet,Stopposisjon bredde,Stopposisjon lengde,Trekkavstand,Rundvekt,Lengdegruppe (kode),Bruttotonnasje Kombinert,Bruttotonnasje Gruppert
353,57.588,6.834,8.0,692,57.534,7.466,38303.0,114.0,4.0,267.0,3
635,61.221,2.181,28.0,330,61.481,1.753,36931.0,100.0,5.0,2053.0,21
1000,58.318,6.210,8.0,71,58.328,6.200,1259.0,10.0,3.0,79.0,1
1001,58.318,6.210,8.0,71,58.328,6.200,1259.0,8.0,3.0,79.0,1
1149,63.411,5.656,7.0,302,63.346,5.699,7558.0,72.0,5.0,536.0,6


In [1707]:
print(df.shape)
df.head()
y = df['Rundvekt'] 
X = df.drop('Rundvekt', axis=1) #Dropper rundvekt pga det er hva jeg trener modellen på



(6054, 11)


In [1709]:
from sklearn.datasets import make_regression
X, y = make_regression(n_samples=200, random_state=1)
# print(X,y)

In [1710]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2) #Test størrelse = 20% av datasettet

In [1711]:
#MinMax Scaler fungerer best med MLP regressor grunnet at verdier som er ekstremt høye/lav kan føre til kluss i loss kalkuleringen

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
# scaler.fit(X)
# X = scaler.transform(X)
X_train_minmax = scaler.fit_transform(X_train)
X_test_minmax = scaler.fit_transform(X_test)

# print(X)

In [1712]:
#Mildt sakt, standard scaler fungerte svært dårlig

# from sklearn.preprocessing import StandardScaler
# sc=StandardScaler()

# scaler = sc.fit(X_train)
# X_train_scaled = scaler.transform(X_train)
# X_test_scaled = scaler.transform(X_test)

In [1713]:
from sklearn.neural_network import MLPRegressor
nn = MLPRegressor(hidden_layer_sizes = [20,10], #Første lag har 20 neurons, andre lag har 10. 
                  activation = 'relu', alpha = 0.1, 
                  max_iter = 2000, #Maks itterasjoner du gjør men det er egentlig epocs. usikker på hvorfor de bruker 'itterations' og not 'epocs'
                #   early_stopping=True, #Early stopping introduserte ustabilitet. men med en iter på 2000 maks så skal det ikke ta alt for lang tid på de fleste pcer.
                  learning_rate_init=0.1,#Start på learning rate, den blir adaptet etter hvert angående på loss verdien
                  learning_rate="adaptive",
                  solver = 'adam', 
                  verbose = False)


In [1714]:

#Test skår uten data leak. (bruker minmax etter splitting)
nn.fit(X_train_minmax, y_train)
print("Neural network, MLPRegressor")
print("Test score",  nn.score(X_test_minmax, y_test))
print("Train  score", nn.score(X_train_minmax, y_train))



Neural network, MLPRegressor
Test score 0.707241424632056
Train  score 0.942746221707945
train minmax [0.64901324 0.70982269 0.54777129 0.2056272  0.13653067 0.55248075
 0.52689285 0.33729798 0.51080482 0.20913965 0.52155672 0.23643919
 0.45259559 0.73214865 0.60134079 0.35528118 0.49752182 0.54760864
 0.65095313 0.48025504 0.59377543 0.42403629 0.7328132  0.93750971
 0.59654453 0.44195873 0.68081658 0.57298198 0.22143087 0.
 0.21793211 0.19208665 0.81682314 0.63981844 0.50328424 0.36491212
 0.47247336 0.07389462 0.31925198 0.69094653 0.4891328  0.4502615
 0.33714277 0.72871916 0.54705959 0.12901696 0.72408443 0.63531096
 0.44529567 0.27132054 0.60139374 0.54242235 0.2838365  0.85928494
 0.37633475 0.53022333 0.37325079 0.26774468 0.52893178 0.45792583
 0.32469858 0.39583691 0.30239706 0.61099672 0.39163714 0.17890702
 0.08823157 0.47185951 0.279634   0.16612655 0.84753618 0.52511866
 0.79577888 0.33409658 0.23696592 0.74323173 0.50376923 0.56704559
 0.3889209  0.64593144 0.43659257 0.


important and relevant properties of the data
how you preprocessed data like which features you selected, did you do dimension reduction,
how you reformatted data, etc.
how you decided on parameters for your machine learning models,
if you used any regularization techniques? In case how.
how the methods were measured and compared


Endlig resultat
Totalt så er det en overfitting problem som skaper problemer men ellers så er resultatene relativt gode. Det er klart ting som kan bli gjort bedre med preprocessing pga skåren noenganger er ganske lav. one-hot encoding kan hjelpe med dette gjennom å få tak i mer data samt med andre strategier som early stopping eller å minske mengen layers eg har (feks fra 2 layers med 20,10 til 1 layer med 7) og cross validation. Ting kan alltid bli forbedret men eg tror eg har nådd punktet der eg kommer til å se diminishing returns.
Metoder ble målt og sammenlignet direkte fra training og test skår. Det var en data leak i skår test metoden helt opp mot slutten men generelt så ville ikke dette ha betydd alt for mye. før og etter eg fikset det så var skårene ganske like.


Dataset features vi valgte
Startposisjon bredde, Startposisjon lengde, Hovedområde start (kode), Varighet, Stopposisjon bredde, Stopposisjon lengde, Trekkavstand, Lengdegruppe (kode), Bruttotonnasje Kombinert, Bruttotonnasje Gruppert
Rundvekt som trenings kolonne.
generelt så ble disse valgt ut i fra testing med og uten verdien (opp til flere ganger for å se gjennomsnitt skår) og sunn fornuft som feks at det blir vansklig å finne rundvekt om du fjerner bruttotonnasje.

Hvordan parameters ble valgt
Ut i fra tester og søking på hva som er best til ett datasett som dette så ble parameterene valgt. Vi startet med en ting som fungerte og snevret inn etter hvert som vi fikset preprossesering.

Vi brukte desverre ingen regularization techniques, men med tanke på at datasettet virker overfitted så ville nok early stopping vært lurt. men med å sette learning rate til "adaptive" så vil datasettet i teorien stoppe når optimum har blitt funnet. ved å introdusere early stopping så klusset det til hele systemet. mest sansynlig grunnet at de klusset med hverandre. Om vi hadde hat mer tid så ville regularization techniques mest sansynlig vis fikset ting og det er en god vei å gå om vi hadde gått videre.





Resultat test breakdown
Test notater eg gjorde mens eg testet modellen. hver test har ca 10 gjennomkjøringer av regressoren med ulike endringer. endringer som var gode ble notert ned.

Første test var bare for å se om ting fungerte riktig eller ikke. letteste måten å gjøre dette på er å fjerne en viktig verdi for å se om det påvirker datasettet på en kraftig måte.
Test 1
Fjernet 'Redskap FDIR (kode)', 'Bruttotonnasje Kombinert',
Test score 0.22802640821841036
Train  score 0.17609209269226367
Eval: Ved å fjerne bruttotonnasje så går test og train skåren ned når du predikerer rundvekt. Dette kommer ikke som en overaskelse pga hvor viktig bruttotonnasje er for å indikere hvor mye fisk en båt kan fange
Datasettet blir lest rett! (før fjerning test=31%, train=90%)

Andre test var ett forsøk på å få skårene litt opp. Å øke iterations (epocs) til 100 000 max førte til bedre skår på kostnadden av en 3 min lang treningsperiode. Noa var klart gale!
Seinere fant eg ut at Standard Scaler ikke fungerer bra med MLPRegressor og når eg hoppet over kunne eg senke iterations til 2000.
Test 2
Låg til scaler, økte iter til 100 000 (kan sikkert senkes til 10 000 etter optimaliseringer)
Test score 0.3805415453308878
Train  score 0.9885294076382664
Klart overtrent men ting blir bedre!

Tredje test var fjerning av repeterende koder og verdier som ikke gav nettverket noen unike verdier å trene på
Test 3
Fjernet: Hovedart - FDIR (kode), Art - FDIR (kode), Art - gruppe (kode)
Test score 0.4551050579085447
Train  score 0.9926394850239878
Minimal økning, det var i alle fall ingen nedtur i nøyaktigheten. men den er enda overfitted


Test 4
Endringer: 
Endret fra standard scaler til minmax grunnet helt sprøe loss verdier som førte til ueffektivitet i antall epocs som kjørte. reduserte fra 10 000-60 000 epocs til ca 2000
Fikset data leak i score kalkuleringen ved å minmax etter splitting
Introduserte adaptiv learning rate.
Endret fra "sdg" til "adam" solver i mlp regressor (slik at eg kan bruke adaptiv learning rate)
Test score 0.707241424632056
Train  score 0.942746221707945
Klart overfitted men har ikke funnet en måte å fikse det på og vi begynner å gå tom for tid. evt ting som kan hjelpe med overfitting er å bruke early stopping før modellen når en optimal loss verdi. Det kan også være at one-hot encoding kan bli brukt for å få bedre data fra strings i datasettet eller annen preprossesering. Det er også mulig å redusere mengden lag og noder som blir skapt. det kan være at 20,10 er for mye (to lag, første med 20 noder, andre med 10)